In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from create_model import create_model

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

model = create_model()
model.summary()

2022-09-13 12:16:50.166392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-13 12:16:50.188180: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-13 12:16:50.188613: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-13 12:16:50.195139: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 625, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 313, 64)      512         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 313, 64)     256         ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 313, 64)      0           ['batch_normalization[0][0]']

In [ ]:
abp = pd.read_csv('data/abp_data.csv', index_col='index')
abp = abp.iloc[1:58652]
abp.drop(labels='sample_id', axis=1, inplace=True)
print(len(abp))

pleth = pd.read_csv('data/pleth_data.csv', index_col='index')
pleth.drop(index=[0, 58652], inplace=True)
pleth.drop(labels='sample_id', axis=1, inplace=True)
print(len(pleth))

labels = np.array([np.array([abp.iloc[i]['sbp'], abp.iloc[i]['dbp']]) for i in range(len(abp))])
print(labels.shape)

windows = np.array([np.array(pleth.iloc[i]).reshape((625, 1)) for i in range(len(pleth))])
print(windows.shape)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((windows, labels))
dataset = dataset.shuffle(buffer_size=1000, seed=69420).batch(32).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

train_ds_size = int(0.70 * len(labels))
val_ds_size = int(0.15 * len(labels))

train_ds = dataset.take(train_ds_size)
remaining = dataset.skip(train_ds_size)  
val_ds = remaining.take(val_ds_size)
test_ds = remaining.skip(val_ds_size)

In [ ]:
model = create_model()
model.summary()
# model.fit(train_ds,
#           validation_data=val_ds,
#           epochs=5)

In [ ]:
from keras.layers import Input
from keras.models import Model
from create_model import resnet

input_shape = (625, 1)
input = Input(shape=input_shape)
output = resnet(input)
model = Model(inputs=input, outputs=output)
model.summary()